In [1]:
import ee
import os
import pandas as pd
import seaborn as sns
import geopandas as gpd
import geemap
import rasterio as rio
import pprint
import time
import gc
import numpy as np

from sklearn.preprocessing import Normalizer
from numpy import unique
from rasterio import features
from osgeo import gdal
from osgeo import osr
from datetime import datetime
from dateutil import relativedelta
from datetime import timedelta


In [2]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [3]:
x = '-56.247947462235125,-7.377467102924857,-55.506370313797625,-7.377467102924857,\
  -55.506370313797625,-6.657806323952262,-56.247947462235125,-6.657806323952262,\
  -56.247947462235125,-7.377467102924857'
  
x = x.split(',')
x = [[float(x[i]), float(x[i+1])] for i in range(0,len(x),2)]


geometria = geometry = ee.Geometry.Polygon(x)

# Definição dos clusters
 - Obteção dos clusters;
 - Criação de mascara para cada cluster obtido;
 - Visualização das mascaras.

In [4]:
# Dataset 
image = ee.ImageCollection("LANDSAT/LC08/C02/T1_RT")\
  .filterBounds(geometry)\
  .filter(ee.Filter.lt('CLOUD_COVER', 5))\
  .filterDate('2012-01-01', '2018-01-01')\
  .median()\
  .clip(geometria)

# Create NDVI index
ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')

# Add NDVI to the image
image = image.addBands(ndvi)

bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI']

image = image.select(bands)
# Select all bands and NDVI for the classification
#print(image.bandNames().getInfo())


# Perform K-means clustering
training_dataset = image.sample(
    **{
        'scale': 30,
        'numPixels': 2000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)


clusters_kmeans = ee.Clusterer.wekaKMeans(9).train(training_dataset)

# Classify the image
classified = image.cluster(clusters_kmeans)

# Create a mask for each of the 8 clusters
mask0 = classified.eq(0)
mask1 = classified.eq(1)
mask2 = classified.eq(2)
mask3 = classified.eq(3)
mask4 = classified.eq(4)
mask5 = classified.eq(5)
mask6 = classified.eq(6)
mask7 = classified.eq(7)
mask8 = classified.eq(8)

In [5]:
# Create a default map
Map = geemap.Map()

#Create an NDWI image, define visualization parameters and display.
col = geemap.get_palette_colors(cmap_name='Dark2',  n_class=7)

masks = [mask0, mask1, mask2, mask3, mask4, mask5, mask6, mask7, mask8]

Map.setCenter(-54.36783956272653,-6.575035169457279,9)


Map.addLayer(image, {'bands':['B4','B3','B2']}, 'RGB')
Map.addLayer(classified.randomVisualizer(), {} , 'teste')
Map.addLayer(masks[0], {'palette': ['white', 'black']}, 'Cluster '+ str(0))
Map.addLayer(masks[1], {'palette': ['white', 'black']}, 'Cluster '+ str(1))
Map.addLayer(masks[2], {'palette': ['white', 'black']}, 'Cluster '+ str(2))
Map.addLayer(masks[3], {'palette': ['white', 'black']}, 'Cluster '+ str(3))
Map.addLayer(masks[4], {'palette': ['white', 'black']}, 'Cluster '+ str(4))
Map.addLayer(masks[5], {'palette': ['white', 'black']}, 'Cluster '+ str(5))
Map.addLayer(masks[6], {'palette': ['white', 'black']}, 'Cluster '+ str(6))
Map.addLayer(masks[7], {'palette': ['white', 'black']}, 'Cluster '+ str(7))
#Map.addLayer(mask0.randomVisualizer(), {} , 'mask')

# Display the map
Map

Map(center=[-6.575035169457279, -54.36783956272653], controls=(WidgetControl(options=['position', 'transparent…

# Definição das dimensões do ***espaço amostral $\Omega$***
*   **Dimensões**:

    * **Espacial**:
    
    ![DEU RUIM](ROI2.png "ROI").

    * **Temporal**: 2012-01-01  → 2022-01-01

*   **Dataset**: Landsat 8

    * **LANDSAT/LC08/C02/T1_RT**         
    

In [6]:
# Dataset 
IMAGE_COLLECTION = ee.ImageCollection("LANDSAT/LC08/C02/T1_RT")\
  .filterBounds(geometry)\
  .filter(ee.Filter.lt('CLOUD_COVER', 80))\
  .filterDate('2012-01-01', '2022-01-01')

## MEDIANA PARA TAPAR BURACO

masked = IMAGE_COLLECTION.map(lambda img: img.updateMask(mask0))

Map = geemap.Map()
Map.setCenter(-54.36783956272653,-6.575035169457279,9)
Map.addLayer(masked.median(), {
  'bands':['B4','B3','B2'],
  'min':100,
  'max':1000
} , 'teste')
Map

Map(center=[-6.575035169457279, -54.36783956272653], controls=(WidgetControl(options=['position', 'transparent…

In [7]:
IMAGE_COLLECTION = IMAGE_COLLECTION.map(lambda img: img.addBands(classified))

In [8]:
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Jdate'] = pd.DatetimeIndex(df['Timestamp']).to_julian_date()
  return df


def fc_to_dict(fc):
  
  prop_names = fc.first().propertyNames()

  prop_lists = fc.reduceColumns(

      reducer=ee.Reducer.toList().repeat(prop_names.size()),

      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)


def create_reduce_region_function(geometry=masked.geometry(),
                                  reducer=ee.Reducer.median(),
                                  scale=30,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):

  def reduce_region_function(img):
    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs,
        bestEffort=bestEffort,
        maxPixels=maxPixels,
        tileScale=tileScale)
    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})  
  return reduce_region_function

def ndvi(imagem):
  ndvi = imagem.expression('(nir - red) / (nir + red)',{
    'nir':imagem.select('B5'),
    'red':imagem.select('B4')}).rename('ndvi')
  return imagem.addBands(ndvi)

def savi(imagem):
  savi = imagem.expression('((nir - red) / (nir + red + 0.5))*(1.5)',
  {'nir':imagem.select('B5'),
  'red':imagem.select('B4')}).rename('savi')
  return imagem.addBands(savi)


IMAGE_COLLECTION = IMAGE_COLLECTION.map(ndvi)
IMAGE_COLLECTION = IMAGE_COLLECTION.map(savi)
IMAGE_COLLECTION = IMAGE_COLLECTION.map(lambda img: img.select(['ndvi','savi']))

In [10]:
for i in range(0,len(masks)):
  
  masked = IMAGE_COLLECTION.map(lambda img: img.updateMask(masks[i]))
  
  reduce_b = create_reduce_region_function(
  geometry = masks[i].geometry(), reducer=ee.Reducer.stdDev(), scale=30)

  b_stat_fc = ee.FeatureCollection(masked.map(reduce_b)).filter(
  ee.Filter.notNull(masked.first().bandNames()))
  
  dict = fc_to_dict(b_stat_fc).getInfo()
  
  df = pd.DataFrame(dict)
  df = add_date_info(df)
  
  name_file = 'df_std'+str(i)+'.csv'
  df.to_csv(name_file)
  print('df'+str(i))

df0
df1
df2
df3
df4
df5
df6
df7
df8


In [11]:
for i in range(0,len(masks)):
  
  masked = IMAGE_COLLECTION.map(lambda img: img.updateMask(masks[i]))
  
  reduce_b = create_reduce_region_function(
  geometry = masks[i].geometry(), reducer=ee.Reducer.median(), scale=30)

  b_stat_fc = ee.FeatureCollection(masked.map(reduce_b)).filter(
  ee.Filter.notNull(masked.first().bandNames()))
  
  dict = fc_to_dict(b_stat_fc).getInfo()
  
  df = pd.DataFrame(dict)
  df = add_date_info(df)
  
  name_file = 'df_median'+str(i)+'.csv'
  df.to_csv(name_file)
  print('df'+str(i))

masked = IMAGE_COLLECTION.map(lambda img: img.updateMask(masks[1]))

In [26]:

#masked = masked.map(lambda img: img.select(['ndvi','savi','B4']))

In [50]:
reduce_b = create_reduce_region_function(geometry=masks[1].geometry(), reducer=ee.Reducer.median(), scale=30)


In [51]:
b_stat_fc = ee.FeatureCollection(masked.map(reduce_b))\
    .filter(ee.Filter.notNull(masked.first().bandNames()))
b_stat_fc

In [ ]:
dict = fc_to_dict(b_stat_fc).getInfo()
dict

EEException: Computation timed out.

In [ ]:

df = pd.DataFrame(dict)
df.sample(7)

In [ ]:
df = add_date_info(df)
df.sample(7)

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
list_ic = masked.toList(masked.size())

In [ ]:
list_ic = masked.toList(masked.size())
newList1 = ee.List([])
dateList = {}
testeDate = {}

Qdias = timedelta(days=30)

def di_to_array(di):
    
    di = di.values()
    di = list(di)
    di = np.array(di)
    
    return di

# Dataset 


for i in list(range(df.index.size)):

    if masked.size().getInfo()!=0:
        
        dateList[i] = df['Jdate'][i]

        blue = 'B2'; green = 'B3'; red = 'B4'; nir = 'B5'; swir1 = 'B6'; swir2 = 'B7';

        def ndvi(imagem):
            ndvi = imagem.expression('(nir - red) / (nir + red)',
            {
                'nir':imagem.select(nir),
                'red':imagem.select(red)
            }).rename('ndvi')
            return imagem.addBands(ndvi)

        def savi(imagem):
            savi = imagem.expression('((nir - red) / (nir + red + 0.5))*(1.5)',
            {
                'nir':imagem.select(nir),
                'red':imagem.select(red)
            }).rename('savi')
            return imagem.addBands(savi)


        masked = masked.map(ndvi)
        masked = masked.map(savi)
        mediana = masked.mean()
        media = masked.median()


        newList1 = newList1.add(mediana)
        newList1 = newList1.add(media)




In [ ]:
def ext_lat_lon_pixel30(image, geometria, bandas):
    image = image.addBands(ee.Image.pixelLonLat())
    coordenadas = image.select(['longitude', 'latitude'] + bandas)\
        .reduceRegion(reducer=ee.Reducer.toList(),
                      geometry=geometria,
                      scale=30,
                      bestEffort=True)

    bandas_valores = []
    for banda in bandas:
        bandas_valores.append(np.array(ee.List(coordenadas.get(banda)).getInfo()).astype(float))

    return np.array(ee.List(coordenadas.get('latitude')).getInfo()).astype(float), np.array(ee.List(coordenadas.get('longitude')).getInfo()).astype(float), bandas_valores

In [ ]:
dateList = di_to_array(dateList)
dateList1 = np.repeat(dateList,2)

In [ ]:
dateList1.size

In [ ]:
trainCollection = ee.ImageCollection(newList1)

In [ ]:
#defaultDummy = 0
#df_ndvi = pd.DataFrame()
#di = {}
#index2 = list(range(0,90))#63
#for j in range(trainCollection.size().getInfo()):
#    tempndvi = index2[j]
#    img = ee.Image(newList1.get(j))
#    lat30, lon30, ind30 = ext_lat_lon_pixel30(img,mask0.geometry(),['ndvi'])
#    di[tempndvi] = ind30[0]
#    
#df_ndvi = df_ndvi.from_dict(di)
#df_ndvi = df_ndvi.assign(Latitude = lat30)
#df_ndvi = df_ndvi.assign(Longitude = lon30)
#df_ndvi = df_ndvi.set_index(['Latitude','Longitude'])
#df_ndvi.sample(10)

In [ ]:
print(f'Shape: {df.shape}\n\n ------------------------------- \n\n col types: {df.dtypes} \n\n \
       size: {IMAGE_COLLECTION.size().getInfo()}')

In [ ]:
sns.lineplot(y='ndvi',
             x='Timestamp',
             data=df)

In [ ]:
from sklearn.preprocessing import Normalizer
# %matplotlib inline: only draw static images in the notebook
subset = ['Timestamp','ndvi','savi','B2','B3','B4','B5']
df = df[subset]
norm = Normalizer()
normalizado = Normalizer().fit_transform(df[['ndvi','savi','B2','B3','B4','B5']].values)
padronizado = pd.DataFrame(normalizado)
padronizado_com_timestap = pd.concat([padronizado,df['Timestamp']],axis=1).rename(columns={i: subset[i] for i in range(0,len(subset))})
padronizado_com_timestap.sample(5)

In [ ]:
ts_plot = padronizado_com_timestap.plot(x=0,
                     kind='line', \
                     title = 'Comportamento espectral ao longo do periodo de análise');
ts_plot.grid()
# define the legend location
ts_plot.legend(loc='upper left');

In [ ]:
sns.lineplot(y='ndvi',
            x='Timestamp',
            data=df)

In [ ]:
padronizado_com_timestap.to_csv('data_frame.csv')

In [ ]:
IMAGE_COLLECTION.add(classified)